<a href="https://colab.research.google.com/github/shengxiaobufu/DL_learning/blob/main/torch/test_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from torch import nn
from torch import Tensor
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [11]:
# create tensor
import numpy as np
t_from_list = torch.tensor([[1.,2], [3,4]])  # by list
t_from_list_int = torch.tensor([[1,2], [3,4]])
t_from_npy_int = torch.tensor(np.array([[1,2], [3,4]])) # numpy
t_from_npy = torch.tensor(np.array([[1.,2], [3,4]]))
print(t_from_list.type())
print(t_from_list_int.type())
print(t_from_npy.type())
print(t_from_npy_int.type())

torch.FloatTensor
torch.LongTensor
torch.DoubleTensor
torch.LongTensor


In [22]:
# move to gpu: copy but not move here
ta2.to(device)
print(ta2.device)
ta3 = ta2.to(device)
print(ta2.device)
print(ta3.device)
# or just init the tensor at target device
t_from_npy = torch.tensor(np.array([[1.,2], [3,4]]), device=device)
print(t_from_npy.device)

cpu
cpu
cuda:0
cuda:0


In [23]:
# use dtype to specify tensor type
t_from_list = torch.tensor([[1.,2], [3,4]], dtype=torch.int32)  # by list
print(t_from_list.type())

torch.IntTensor


In [26]:
# init in target dim
t_zero = torch.zeros([2, 1])
print(t_zero.type(), t_zero)
t_one = torch.ones([2, 1])
print(t_one.type(), t_one)
t_one = torch.ones([2, 1], dtype=torch.int32)
print(t_one.type(), t_one)
t_one = torch.ones([2, 1, 3], dtype=torch.float64)
print(t_one.type(), t_one)

torch.FloatTensor tensor([[0.],
        [0.]])
torch.FloatTensor tensor([[1.],
        [1.]])
torch.IntTensor tensor([[1],
        [1]], dtype=torch.int32)
torch.DoubleTensor tensor([[[1., 1., 1.]],

        [[1., 1., 1.]]], dtype=torch.float64)


In [36]:
# change dim
t_one = torch.ones([2, 1, 3, 1], dtype=torch.float64, device=device)
print(t_one)

tensor([[[[1.],
          [1.],
          [1.]]],


        [[[1.],
          [1.],
          [1.]]]], device='cuda:0', dtype=torch.float64)


In [35]:
# slice of tensor, almost the same as numpy
print(t_one[0])
print(t_one[1][0])
print(t_one[0, 0, :2])
print(t_one[0][0][0][0])

tensor([[[1.],
         [1.],
         [1.]]], device='cuda:0', dtype=torch.float64)
tensor([[1.],
        [1.],
        [1.]], device='cuda:0', dtype=torch.float64)
tensor([[1.],
        [1.]], device='cuda:0', dtype=torch.float64)
tensor(1., device='cuda:0', dtype=torch.float64)


In [41]:
# covert to pure number, use .item(), only allow one item, but not limit for one dim
print(t_one[0][0][0][0])
print(t_one[0][0][0][0].item())
print(t_one[0][0][0])
print(t_one[0][0][0].item())
print(t_one[0][0])
# print(t_one[0][0].item())   # RuntimeError: a Tensor with 3 elements cannot be converted to Scalar

tensor(1., device='cuda:0', dtype=torch.float64)
1.0
tensor([1.], device='cuda:0', dtype=torch.float64)
1.0
tensor([[1.],
        [1.],
        [1.]], device='cuda:0', dtype=torch.float64)


In [46]:
# use autograd to enable grad calculate
'''
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=False)
out = x.pow(2).sum()
out.backward()
# without set requires_grad=True, backward of functions of x is not usable
# RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn
'''
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
out = x.pow(2).sum()
out.backward()
print(x.grad)  # 这里返回的就是对x的一阶偏导数, d(x**2)/dx = 2x


tensor([[ 2., -2.],
        [ 2.,  2.]])


In [58]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)

out = (2*x.pow(2) + x).sum()
out.backward()
print(x.grad)  # 这里返回的就是对x的一阶偏导数, d(2x)/dx = 2*2*x + 1

tensor([[ 5., -3.],
        [ 5.,  5.]])


In [64]:
'''
A torch.layout is an object that represents the memory layout of a torch.Tensor.
 Currently, we support torch.strided (dense Tensors) and have beta support for
 torch.sparse_coo (sparse COO Tensors).
Strides are a list of integers: the k-th stride represents the jump in the memory
 necessary to go from one element to the next one in the k-th dimension of the Tensor.
 This concept makes it possible to perform many tensor operations efficiently.
'''
print(x.layout)
print(x.stride())

torch.strided
(2, 1)


In [69]:
x = torch.tensor([[1., 21, 31], [2., 22, 32]], requires_grad=True)
print(x.layout)
print(x.stride())
# 按照dim1遍历，从1到2需要经过3个内存位置，按照dim2遍历，从1到21只需要经过一个内存位置
# 在tensor特别大的时候通过这个来判断指针如何移动，提高效率

torch.strided
(3, 1)
